In [1]:
from code import *
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import time
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Lambda, Compose



Création du dataset et chargement dans le DataLoader
la fonction `gen_csv(img_dir, path_csv)` génère le fichier csv des labels selon les dossier des images

In [2]:
CONFIG = {
    "label_dir": "./labels",
    "img_dir": "./img",
    "optimizer":"SGD",
    "learning_rate":0.01,
    "epoch":50,
    "loss":"cross_entropy",
    "batch_size":16,
    "device":"cuda" if torch.cuda.is_available() else "cpu",
    "path_config":"./config.json", # PATH TO CHANGE
    "loss_history" : {"train":[],
                      "val": [],
                      },
    "overall_time": 0,
    "transforms": transforms.Compose([
        transforms.Resize((480,640)),
    ])
}

In [3]:
## Dataset
data = CustomImageDataset(CONFIG["label_dir"], CONFIG["img_dir"], transform=CONFIG["transforms"])

## DataLoader
test_prop = 0.1
val_prop = 0.1
data, test_set = random_split(data, [1 - test_prop, test_prop])
train_set, val_set = random_split(data, [1 - val_prop, val_prop])
train_dataloader = DataLoader(train_set, batch_size=CONFIG["batch_size"], shuffle=True)
val_dataloader = DataLoader(val_set, batch_size=CONFIG["batch_size"], shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=CONFIG["batch_size"], shuffle=True)
CONFIG["n_train"] = len(train_dataloader)
CONFIG["n_val"] = len(val_dataloader)

## Network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.stack_conv_relu_pool = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5, stride=3, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.stack_fullyconnected = nn.Sequential(
            nn.Flatten(),
            nn.Linear(8320, 640),
            nn.ReLU(),
            nn.Linear(640, 107),
            nn.ReLU(),
            nn.Linear(107, 18),
            nn.ReLU(),
            nn.Linear(18, 3),
        )

    def forward(self, x):
        x = self.stack_conv_relu_pool(x)
        logits = self.stack_fullyconnected(x)
        return logits

net = Net().to(CONFIG["device"])
print("\n")
print(net)
nombre_parametres = sum(p.numel() for p in net.parameters())
print(f"Nombre de paramètres : {nombre_parametres}")

criterion = torch.nn.CrossEntropyLoss().to(CONFIG["device"])

optimizer = optim.SGD(net.parameters(), lr=CONFIG["learning_rate"])



                             name  label
0              nobody/nobody0.jpg      0
1           nobody/nobody1000.jpg      0
2           nobody/nobody1001.jpg      0
3           nobody/nobody1002.jpg      0
4           nobody/nobody1003.jpg      0
...                           ...    ...
2178     people/peopleimage10.jpg      2
2179     people/peopleimage11.jpg      2
2180  people/peopleimagecompr.jpg      2
2181      people/peopletest12.jpg      2
2182             people/peopletrj      2

[2183 rows x 2 columns]


Net(
  (stack_conv_relu_pool): Sequential(
    (0): Conv2d(3, 8, kernel_size=(5, 5), stride=(3, 3), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()


# Training loop

In [4]:
for e in range(CONFIG["epoch"]):

    #--- Init ---#
    train_loss = 0
    val_loss = 0
    clk = time.time()
    
    #--- Training ---#
    net.train()
    for data in train_dataloader:

        #--- get image and label from data ---#
        train_features, train_labels = data
        train_features = train_features / 255
        if torch.cuda.is_available():
          train_features, train_labels = train_features.cuda(), train_labels.cuda()
        #--- reset optimizer ---#
        optimizer.zero_grad()
        #--- prediction ---#
        pred = net(train_features)
        #--- loss function ---#
        loss = criterion(pred, train_labels)
        #--- backpropagation ---#
        loss.backward()
        #--- Network optimization ---#
        optimizer.step()
        train_loss += loss.item()

    #--- Validation ---#
    net.eval()
    with torch.no_grad():
        for data in val_dataloader:

            #--- get image and label from data ---#
            val_features, val_labels = next(iter(val_dataloader))
            val_features = val_features / 255
            if torch.cuda.is_available():
                val_features, val_labels = val_features.cuda(), val_labels.cuda()
            #--- prediction ---#
            pred = net(val_features)
            #--- loss function ---#
            loss = criterion(pred, val_labels)
            val_loss += loss.item()

    #--- Epoch results ---#
    train_loss /= CONFIG["n_train"]
    val_loss /= CONFIG["n_val"]
    clk = time.time() - clk
    CONFIG["overall_time"] += clk
    CONFIG["loss_history"]["train"].append(train_loss)
    CONFIG["loss_history"]["val"].append(val_loss)
    print('Epoch {}: Train loss = avg {:0.4f} || Valid loss = avg {:0.4f}\nTime :{:0.1f}mn {:0.1f}s\n'.format(e,train_loss,val_loss,clk//60,clk%60))

C:\Users\asury\Documents\Cours\3A\Master\projet\venv\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 0: Train loss = avg 1.0410 || Valid loss = avg 0.9898
Time :4.0mn 8.5s
Epoch 1: Train loss = avg 0.9424 || Valid loss = avg 0.8973
Time :3.0mn 30.4s
Epoch 2: Train loss = avg 0.9028 || Valid loss = avg 0.9081
Time :4.0mn 39.9s
Epoch 3: Train loss = avg 0.8875 || Valid loss = avg 0.9361
Time :1.0mn 41.8s
Epoch 4: Train loss = avg 0.8788 || Valid loss = avg 0.8919
Time :2.0mn 5.0s
Epoch 5: Train loss = avg 0.8760 || Valid loss = avg 0.9473
Time :2.0mn 9.4s
Epoch 6: Train loss = avg 0.8730 || Valid loss = avg 0.9635
Time :1.0mn 27.2s
Epoch 7: Train loss = avg 0.8718 || Valid loss = avg 0.9176
Time :1.0mn 43.4s
Epoch 8: Train loss = avg 0.8698 || Valid loss = avg 0.8691
Time :1.0mn 24.0s
Epoch 9: Train loss = avg 0.8700 || Valid loss = avg 0.9034
Time :1.0mn 23.6s
Epoch 10: Train loss = avg 0.8693 || Valid loss = avg 0.8755
Time :1.0mn 31.3s
Epoch 11: Train loss = avg 0.8683 || Valid loss = avg 0.9483
Time :1.0mn 23.7s
Epoch 12: Train loss = avg 0.8684 || Valid loss = avg 0.8307
Time

In [6]:
torch.save(net.state_dict(),"./train50.pt")

In [ ]:


#------------------------------------------------------------------------------------------------------------------------------------------------#
def plot_training_curves(config):
    """Save training curves of a network."""

    #--- Init ---#
    n_epoch = config["epoch"]
    epochs = np.arange(1,n_epoch+1)
    fig = plt.figure(figsize=(19.2,10.8),dpi=100)

    #--- Plot ---#
    plt.plot(epochs,config["loss_history"]["train"],label="Train")
    plt.plot(epochs,config["loss_history"]["val"],label="Val")
    plt.title("Loss curves")
    plt.legend()
    plt.show()
#------------------------------------------------------------------------------------------------------------------------------------------------#

#------------------------------------------------------------------------------------------------------------------------------------------------#
def evaluate_model(net,test_dataloader,criterion,config):
  """Evaluate the model on a given test set."""

  #--- Init ---#
  test_loss = 0
  count = 0
  n_test = len(test_dataloader)
  config["n_test"] = n_test

  #--- Evaluate ---#
  net.eval()
  with torch.no_grad():
    for data in test_dataloader:

      #--- get image and label from data ---#
      data, labels = data
      data = data / 255
      if torch.cuda.is_available():
        data, labels = data.cuda(), labels.cuda()
      #--- prediction ---#
      pred = net(data)
      #--- loss function ---#
      loss = criterion(pred, labels)
      #--- accuracy ---#
      y = torch.argmax(pred)
      count += (y == labels.item())

  #--- Results ---#
  count = count / config["n_test"]
  count *= 100
  config["loss_history"]["test"] = count.item()
  print("Test accuracy: {:0.2f}%".format(count))

  return config
#------------------------------------------------------------------------------------------------------------------------------------------------#

""" Plot training curves """
#------------------------------------------------------------------------------------------------------------------------------------------------#
plot_training_curves(CONFIG)
#------------------------------------------------------------------------------------------------------------------------------------------------#

""" Evaluate network on test set """
#------------------------------------------------------------------------------------------------------------------------------------------------#
CONFIG = evaluate_model(net,test_dataloader,criterion,CONFIG)
#------------------------------------------------------------------------------------------------------------------------------------------------#

""" Save results """
#------------------------------------------------------------------------------------------------------------------------------------------------#
#--- Save CONFIG ---#
with open("save_config.json",'w') as f:
    json.dump(CONFIG, f, indent=4)

#--- Save training curves ---#
# ... #
#------------------------------------------------------------------------------------------------------------------------------------------------#